## Build a classification model for Survival.csv dataset
Find dataset describtion here
https://physionet.org/content/challenge-2012/1.0.0/


In [1]:
# Create train.py which will return the table of model results
# Create a table to compere all classification models LR, LDA, QDA, DT, NB, SVM, Ansamble models 
# With this metrics, Accuracy, Sensitifity, Specificity, AUC, MCC(key for assesment)
# Save the model in pickle format.
# Create predict.py file which will load the model and predict on the test.csv file
# Your code should be able to run from the command line having test argument
# And should save the predictions.npy which are the probabilities of the class being true.
# Ex. python predict.py --test test.csv 

# Columns <survival, length_of_stay and died_in_hospital" will not appear in test data

In [78]:
import pyforest
import sys
from impyute.imputation.cs import fast_knn
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import f1_score, matthews_corrcoef, classification_report, accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import pickle

In [3]:
df = pd.read_csv('Survival_dataset.csv')
df= df.drop(['Survival', 'Length_of_stay'], axis=1)

<IPython.core.display.Javascript object>

In [4]:
# fill missing values with kNN, k=5 (5 is random number)
imputed_training=fast_knn(df.values, k=5)

In [5]:
df_filled_kNN = pd.DataFrame(data = imputed_training, columns = df.columns)

<IPython.core.display.Javascript object>

In [6]:
df_filled_kNN.head()

,recordid,SAPS-I,SOFA,In-hospital_death,Age,Gender,Height,Weight,CCU,CSRU,...,SysABP_last,TroponinI_last,TroponinT_last,WBC_last,Weight_last,pH_last,MechVentStartTime,MechVentDuration,MechVentLast8Hour,UrineOutputSum
0,132539.0,6.0,1.0,0.0,54.0,0.0,169.069577,74.186860,0.0,0.0,...,127.660441,6.466341,0.806923,9.4,79.891331,7.397115,285.17675,1912.00514,0.620799,12.314749
1,132540.0,16.0,8.0,0.0,76.0,1.0,175.300000,76.000000,0.0,1.0,...,103.000000,6.466341,1.002283,13.3,81.600000,7.370000,71.00000,360.00000,0.000000,5.000000
2,132541.0,21.0,11.0,0.0,44.0,0.0,168.829614,56.700000,0.0,0.0,...,126.000000,6.466341,0.824006,6.2,56.700000,7.470000,617.00000,2160.00000,1.000000,14.000000
3,132543.0,7.0,1.0,0.0,68.0,1.0,180.300000,84.600000,0.0,0.0,...,129.497189,6.466341,1.002283,7.9,84.600000,7.393791,285.17675,1912.00514,0.620799,12.314749
4,132545.0,17.0,2.0,0.0,88.0,0.0,168.890792,75.941738,0.0,0.0,...,127.200042,6.466341,1.002283,4.8,84.545159,7.399185,285.17675,1912.00514,0.620799,12.314749


In [26]:
x = df_filled_kNN[df_filled_kNN.columns.difference(['In-hospital_death'])]
y = df_filled_kNN['In-hospital_death']

In [27]:
#small test of feature importance with decision tree
model = DecisionTreeClassifier()
model.fit(x, y)
importance = model.feature_importances_
imp_zip = zip(np.arange(0, len(importance), 1), importance)
importance_sorted = sorted(imp_zip, key = lambda x: x[1], reverse=True)

<IPython.core.display.Javascript object>

In [28]:
selected_features_indxs = [i for i, j in importance_sorted if j>0.005]

In [29]:
filtered_x = x.iloc[:, selected_features_indxs]

In [30]:
filtered_x.shape

(4000, 72)

In [31]:
#x=filtered_x

In [32]:
#fitting all models, except for ensemble models
lr = LogisticRegression()
lr.fit(x, y)
y_pred_lr = lr.predict(x)
lda = LinearDiscriminantAnalysis()
lda.fit(x, y)
y_pred_lda = lda.predict(x)
qda = QuadraticDiscriminantAnalysis()
qda.fit(x, y)
y_pred_qda = qda.predict(x)
dt = DecisionTreeClassifier()
dt.fit(x, y)
y_pred_dt = dt.predict(x)
nb = GaussianNB()
nb.fit(x, y)
y_pred_nb = nb.predict(x)
svm = SVC()
svm.fit(x, y)
y_pred_svm = nb.predict(x)

C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [34]:
cl_report_lr = classification_report(y, y_pred_lr, output_dict=True)
cl_report_lda = classification_report(y, y_pred_lda, output_dict=True)
cl_report_qda = classification_report(y, y_pred_qda, output_dict=True)
cl_report_dt = classification_report(y, y_pred_dt, output_dict=True)
cl_report_nb = classification_report(y, y_pred_nb, output_dict=True)
cl_report_svm = classification_report(y, y_pred_svm, output_dict=True)

In [35]:
scores = {"Logistic Regression" : {"precision" : cl_report_lr['weighted avg']['precision'],
                                   "recall" :  cl_report_lr['weighted avg']['recall'],
                                   "f1_score" :  cl_report_lr['weighted avg']['f1-score']},
          "LDA" : {"precision" : cl_report_lda['weighted avg']['precision'],
                                   "recall" :  cl_report_lda['weighted avg']['recall'],
                                   "f1_score" :  cl_report_lda['weighted avg']['f1-score']},
          "QDA" : {"precision" : cl_report_qda['weighted avg']['precision'],
                                   "recall" :  cl_report_qda['weighted avg']['recall'],
                                   "f1_score" :  cl_report_qda['weighted avg']['f1-score']},
          "Decision Tree" : {"precision" : cl_report_dt['weighted avg']['precision'],
                                   "recall" :  cl_report_dt['weighted avg']['recall'],
                                   "f1_score" :  cl_report_dt['weighted avg']['f1-score']},
          "Naive Bayes" : {"precision" : cl_report_nb['weighted avg']['precision'],
                                   "recall" :  cl_report_nb['weighted avg']['recall'],
                                   "f1_score" :  cl_report_nb['weighted avg']['f1-score']},
          "SVM" : {"precision" : cl_report_svm['weighted avg']['precision'],
                                   "recall" :  cl_report_svm['weighted avg']['recall'],
                                   "f1_score" :  cl_report_svm['weighted avg']['f1-score']}
    
}

In [36]:
scores_table = pd.DataFrame(scores).T

<IPython.core.display.Javascript object>

In [37]:
print(scores_table)

                     precision   recall  f1_score
Logistic Regression   0.847123  0.87175  0.838367
LDA                   0.865629  0.88200  0.867672
QDA                   0.895800  0.89450  0.895128
Decision Tree         1.000000  1.00000  1.000000
Naive Bayes           0.834045  0.81975  0.826266
SVM                   0.834045  0.81975  0.826266


In [68]:
#trying to get more generalized scores with cross validation
LR = LogisticRegression()
LDA = LinearDiscriminantAnalysis()
QDA = QuadraticDiscriminantAnalysis()
DT = DecisionTreeClassifier()
NB = GaussianNB()
SVM = SVC()

In [72]:
cv = KFold(n_splits=10, shuffle=True, random_state=142)
scoring = ['precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']

In [73]:
LR_scores = cross_validate(LR,x, y, scoring=scoring, cv=cv)
LDA_scores = cross_validate(LDA, x, y,scoring=scoring, cv=cv)
QDA_scores = cross_validate(QDA, x, y,scoring=scoring, cv=cv)
DT_scores = cross_validate(DT, x, y,scoring=scoring, cv=cv)
NB_scores = cross_validate(NB, x, y,scoring=scoring, cv=cv)
SVM_scores = cross_validate(SVM, x, y, scoring=scoring,cv=cv)

C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\user\Downloads\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
 

In [76]:
scores_cv = {"Logistic Regression" : {"precision" : np.mean(LR_scores['test_precision_weighted']),
                                   "recall" :  np.mean(LR_scores['test_recall_weighted']),
                                   "f1_score" :  np.mean(LR_scores['test_f1_weighted']),
                                    "AUC" :  np.mean(LR_scores['test_roc_auc'])},
        "LDA" : {"precision" : np.mean(LDA_scores['test_precision_weighted']),
                                   "recall" :  np.mean(LDA_scores['test_recall_weighted']),
                                   "f1_score" :  np.mean(LDA_scores['test_f1_weighted']),
                                    "AUC" :  np.mean(LDA_scores['test_roc_auc'])},
        "QDA" : {"precision" : np.mean(QDA_scores['test_precision_weighted']),
                                   "recall" :  np.mean(QDA_scores['test_recall_weighted']),
                                   "f1_score" :  np.mean(QDA_scores['test_f1_weighted']),
                                    "AUC" :  np.mean(QDA_scores['test_roc_auc'])},
        "Decision tree" : {"precision" : np.mean(DT_scores['test_precision_weighted']),
                                   "recall" :  np.mean(DT_scores['test_recall_weighted']),
                                   "f1_score" :  np.mean(DT_scores['test_f1_weighted']),
                                    "AUC" :  np.mean(DT_scores['test_roc_auc'])},
        "Naive Bayes" : {"precision" : np.mean(NB_scores['test_precision_weighted']),
                                   "recall" :  np.mean(NB_scores['test_recall_weighted']),
                                   "f1_score" :  np.mean(NB_scores['test_f1_weighted']),
                                    "AUC" :  np.mean(NB_scores['test_roc_auc'])},
        "SVM" : {"precision" : np.mean(SVM_scores['test_precision_weighted']),
                                   "recall" :  np.mean(SVM_scores['test_recall_weighted']),
                                   "f1_score" :  np.mean(SVM_scores['test_f1_weighted']),
                                    "AUC" :  np.mean(SVM_scores['test_roc_auc'])},
        
}

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [77]:
scores_cv_table = pd.DataFrame(scores_cv).T
print(scores_cv_table)

<IPython.core.display.Javascript object>

                     precision   recall  f1_score       AUC
Logistic Regression   0.831347  0.86450  0.832575  0.814029
LDA                   0.849643  0.86825  0.851861  0.834198
QDA                   0.822219  0.83175  0.825424  0.759247
Decision tree         0.811830  0.80475  0.807625  0.605941
Naive Bayes           0.830456  0.81300  0.820225  0.790961
SVM                   0.742610  0.86150  0.797537  0.500000


In [80]:
#saving the model / here it's also loaded but that part should be in predict.py file /
best_model_LDA = LinearDiscriminantAnalysis()
best_model_LDA.fit(x, y)
file = 'best_model.pkl'
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model_LDA, file)
with open('best_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [82]:
model.score(x, y)

0.882